In [1]:
from functools import reduce
from math import log, exp, inf
from itertools import product
from random import random, randint, seed, choice
import numpy as np
from scipy import sparse
from copy import copy

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets


In [2]:
x = None

In [4]:
def tweak(state):
    new_state = copy(state)
    i = randint(0, len(state) - 1)
    new_state[i] = not new_state[i]
    return new_state

def check_tabu_presence(sol, tabu_list):
    return ((sol == tabu_list).all()).any()

def fit_diff(fit1, fit2):
    return fit1[0] + fit1[1] - fit2[0] - fit2[1]

def log_schedule(init_temp, step):
    return init_temp / (1 + log(1 + step))

def lin_schedule(init_temp, step):
    return init_temp / (1 + step)

def is_valid(state):
    return np.all(
        reduce(
            np.logical_or,
            [x[[i], :].toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(state))]),
        )
    )

def fitness1(state):
    cost = sum(state)
    filled = np.sum(
        reduce(
            np.logical_or,
            [x[[i], :].toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(state))]),
        )
    )
    return filled, -cost

def fitness2(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [x[[i], :].toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(state))]),
        )
    )
    return valid, -cost

In [16]:
def first_improvement(state, fitness):
    s =  state.copy()
    fit_s = fitness(s)
    n_calls = 1
    steady = 0
    for step in range(5000):
        r = tweak(s.copy())
        fit_r = fitness(r)
        n_calls += 1
        if fit_r > fit_s:
            s = r
            fit_s = fit_r
            steady = 0
        else:
            steady += 1
        if steady == 500:
            print(f"Interruption at the {step}-th step")
            break
    return s, n_calls

In [9]:
def steepest_step_hc(state, n_samples, fitness):
    s = state.copy()
    fit_s = fitness(s)
    steady = 0
    n_calls = 1
    for step in range(5000):
        r = tweak(s.copy())
        fit_r = fitness(r)
        n_calls += 1
        for _ in range(n_samples - 1):
            w = tweak(s.copy())
            fit_w = fitness(w)
            n_calls += 1
            if fit_w > fit_r:
                r = w
                fit_r = fit_w
        if fit_r > fit_s:
            s = r
            fit_s = fit_r
            steady = 0
        else:
            steady += 1
        if steady == 500:
            print(f"Interruption at the {step}-th step")
            break
    return s, n_calls

In [4]:
def simulated_annealing(state, init_temp, schedule, fitness):
    s = state.copy()
    fit_s = fitness(s)
    steady = 0
    num_calls = 1
    for step in range(5000):
        r = tweak(s.copy())
        fit_r = fitness(r)
        num_calls += 1
        if fit_r > fit_s:   #tweaked state is better
            s = r
            fit_s = fit_r
            steady = 0
        else:    #do I accept the worse solution?
            t = schedule(init_temp, step)
            p = exp(-fit_diff(fit_s, fit_r)/t)
            # print(f"prob di accettare: {p}")
            if random() <= p:
                s = r
                fit_s = fit_r
                steady = 0
            else:
                steady += 1
        if steady == 500:
            print(f"Interruption at the {step}-th step")
            break
    return s, num_calls


In [5]:
def tabu_search(state, n_samples, fitness):
    max_size = 500
    best_cand = state.copy()
    best = state.copy()
    steady = 0
    tabu_list = []
    tabu_list.append(best_cand)
    num_calls = 0
    for step in range(5000):
        fit_best_cand = (-inf, -inf)
        tmp = [tweak(best_cand) for _ in range(n_samples)]
        cand = [sol for sol in tmp if not check_tabu_presence(sol, tabu_list)]
        for c in cand:
            f = fitness(c)
            num_calls += 1
            if f > fit_best_cand:
                best_cand = c
                fit_best_cand = f
        if fit_best_cand == (-inf, -inf):
            break
        fb = fitness(best)
        num_calls += 1
        if fit_best_cand > fb:
            fb = fit_best_cand
            best = best_cand
            steady = 0
        else:
            steady += 1
        if steady == 500:
            print(f"Stop at the {step}-th step")
            break
        tabu_list.append(best_cand)
        if len(tabu_list) > max_size:
            tabu_list.pop(0)
        
    return best, num_calls
        

In [10]:
print("Steepest-step")
for ds in [.3, .7]:
    for ns in [100, 1_000, 5_000]:
        x = make_set_covering_problem(ns, ns, ds)
        init_state = np.array([False for _ in range(ns)])
        f_state, n_calls = steepest_step_hc(init_state.copy(), 10, fitness1)
        print(f"density: {ds}, num_sets: {ns}")
        print(f"Acceptable solution: {is_valid(f_state)}, #tiles: {sum(f_state)}")
        print(f"Number of calls to fitness function: {n_calls}")

Steepest-step
Interruption at the 507-th step
density: 0.3, num_sets: 100
Acceptable solution: True, #tiles: 8
Number of calls to fitness function: 5081
Interruption at the 511-th step
density: 0.3, num_sets: 1000
Acceptable solution: True, #tiles: 12
Number of calls to fitness function: 5121
Interruption at the 518-th step
density: 0.3, num_sets: 5000
Acceptable solution: True, #tiles: 18
Number of calls to fitness function: 5191
Interruption at the 502-th step
density: 0.7, num_sets: 100
Acceptable solution: True, #tiles: 3
Number of calls to fitness function: 5031
Interruption at the 504-th step
density: 0.7, num_sets: 1000
Acceptable solution: True, #tiles: 5
Number of calls to fitness function: 5051
Interruption at the 505-th step
density: 0.7, num_sets: 5000
Acceptable solution: True, #tiles: 6
Number of calls to fitness function: 5061


In [8]:
print("Simulated_annealing")
for ds in [.3, .7]:
    for ns in [100, 1_000, 5_000]:
        x = make_set_covering_problem(ns, ns, ds)
        init_state = np.array([False for _ in range(ns)])
        f_state, n_calls = simulated_annealing(init_state.copy(), 20, lin_schedule, fitness1)
        print(f"density: {ds}, num_sets: {ns}")
        print(f"Acceptable solution: {is_valid(f_state)}, #tiles: {sum(f_state)}")
        print(f"Number of calls to fitness function: {n_calls}")

Simulated_annealing
Interruption at the 1074-th step
density: 0.3, num_sets: 100
Acceptable solution: True, #tiles: 7
Number of calls to fitness function: 1076
Interruption at the 1510-th step
density: 0.3, num_sets: 1000
Acceptable solution: True, #tiles: 13
Number of calls to fitness function: 1512
Interruption at the 580-th step
density: 0.3, num_sets: 5000
Acceptable solution: True, #tiles: 28
Number of calls to fitness function: 582
Interruption at the 1300-th step
density: 0.7, num_sets: 100
Acceptable solution: True, #tiles: 3
Number of calls to fitness function: 1302
Interruption at the 1383-th step
density: 0.7, num_sets: 1000
Acceptable solution: True, #tiles: 7
Number of calls to fitness function: 1385
Interruption at the 686-th step
density: 0.7, num_sets: 5000
Acceptable solution: True, #tiles: 17
Number of calls to fitness function: 688


In [6]:
print("Tabu search")
for ds in [.3, .7]:
    for ns in [100, 1_000, 5_000]:
        x = make_set_covering_problem(ns, ns, ds)
        init_state = np.array([False for _ in range(ns)])
        f_state, n_calls = tabu_search(init_state.copy(), 10, fitness1)
        print(f"density: {ds}, num_sets: {ns}")
        print(f"Acceptable solution: {is_valid(f_state)}, #tiles: {sum(f_state)}")
        print(f"Number of calls to fitness function: {n_calls}")

Tabu search
Stop at the 507-th step
density: 0.3, num_sets: 100
Acceptable solution: True, #tiles: 8
Number of calls to fitness function: 5588
Stop at the 511-th step
density: 0.3, num_sets: 1000
Acceptable solution: True, #tiles: 12
Number of calls to fitness function: 5632
Stop at the 518-th step
density: 0.3, num_sets: 5000
Acceptable solution: True, #tiles: 19
Number of calls to fitness function: 5709
Stop at the 502-th step
density: 0.7, num_sets: 100
Acceptable solution: True, #tiles: 3
Number of calls to fitness function: 5533
Stop at the 504-th step
density: 0.7, num_sets: 1000
Acceptable solution: True, #tiles: 5
Number of calls to fitness function: 5555
Stop at the 505-th step
density: 0.7, num_sets: 5000
Acceptable solution: True, #tiles: 6
Number of calls to fitness function: 5566


In [17]:
print("First improvement")
for ds in [.3, .7]:
    for ns in [100, 1_000, 5_000]:
        x = make_set_covering_problem(ns, ns, ds)
        init_state = np.array([False for _ in range(ns)])
        f_state, n_calls = first_improvement(init_state.copy(), fitness1)
        print(f"density: {ds}, num_sets: {ns}")
        print(f"Acceptable solution: {is_valid(f_state)}, #tiles: {sum(f_state)}")
        print(f"Number of calls to fitness function: {n_calls}")

First improvement
Interruption at the 683-th step
density: 0.3, num_sets: 100
Acceptable solution: True, #tiles: 8
Number of calls to fitness function: 685
Interruption at the 596-th step
density: 0.3, num_sets: 1000
Acceptable solution: True, #tiles: 16
Number of calls to fitness function: 598
Interruption at the 523-th step
density: 0.3, num_sets: 5000
Acceptable solution: True, #tiles: 22
Number of calls to fitness function: 525
Interruption at the 728-th step
density: 0.7, num_sets: 100
Acceptable solution: True, #tiles: 3
Number of calls to fitness function: 730
Interruption at the 505-th step
density: 0.7, num_sets: 1000
Acceptable solution: True, #tiles: 6
Number of calls to fitness function: 507
Interruption at the 592-th step
density: 0.7, num_sets: 5000
Acceptable solution: True, #tiles: 7
Number of calls to fitness function: 594
